In [34]:
import pandas as pd
import numpy as np
import random
import math
from typing import Dict, List, Tuple

import plotly.express as px
pd.options.plotting.backend = "plotly"

In [35]:
#market behavior simulation functions
def short_sin(day:int, cycle:int):
    value = 1.5 + (0.5 * math.sin((day - 7.5) / (cycle / (2*math.pi))))
    return value


def short_cos(day:int, cycle:int):
    value = 1.5 + (0.5 * math.cos((day + cycle) / (cycle / (2*math.pi))))
    return value


def long_sin(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


def long_cos(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.cos(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


#reward rate framework
def rr_framework(supply:int):
    if supply > 1000000000:
      return 0.001
    elif supply > 100000000:
      return 0.002
    else:
      return 0.004

In [36]:
class ModelParams():
    def __init__(self, horizon:int, max_liq_ratio:float, demand_factor:float, supply_factor:float, short_cycle:int, long_cycle:int, long_sin_offset:float, long_cos_offset:float, initial_target:float):
        self.horizon = horizon
        self.max_liq_ratio = max_liq_ratio
        self.demand_factor = demand_factor
        self.supply_factor = supply_factor
        self.short_cycle = short_cycle
        self.long_cycle = long_cycle
        self.long_sin_offset = long_sin_offset
        self.long_cos_offset = long_cos_offset
        self.initial_target = initial_target


class Day():
    def __init__(self, params:ModelParams, prev_arbs:Dict[int, Tuple[float, float]], prev_lags=Dict[int, Tuple[int, Dict[int, float]]], prev_day=None):

        if prev_day is None:
            self.day = 1
            self.supply = 11000000
            self.reward_rate = rr_framework(self.supply)
            self.price = 46
            self.natural_target = params.initial_target
            self.real_target = params.initial_target
            self.liq_usd = 90000000
            self.liq_ohm = self.liq_usd / self.price
            self.k = (self.liq_usd ** 2) / self.price
            self.reserves_in = 0
            self.reserves_out = 0
            self.reserves = 244000000
            self.market_demand = params.demand_factor
            self.market_supply = params.supply_factor
            self.arb_factor = 0
            self.arb_demand = 0
            self.arb_supply = 0
            self.unwind_demand = 0
            self.unwind_supply = 0
            self.net_flow = random.uniform(self.liq_usd * self.market_supply, self.liq_usd * self.market_demand)
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)

        else:
            self.day = prev_day.day + 1
            self.reward_rate = rr_framework(prev_day.supply)
            self.supply = prev_day.supply * (1 + self.reward_rate)
            self.natural_target = calc_natural_target(short_cycle=params.short_cycle, prev_day=prev_day)
            self.real_target = calc_price_target(short_cycle=params.short_cycle, prev_day=prev_day, prev_lags=prev_lags)

            #AMM k
            if prev_day.liq_ratio < params.max_liq_ratio:
                self.k = prev_day.k * (1 + self.reward_rate)**2
            else:
                self.k = prev_day.k * (params.max_liq_ratio * (1 + self.reward_rate)**2) / (prev_day.liq_ratio)

            #reserves in
            if params.max_liq_ratio * prev_day.liq_usd > prev_day.reserves * (1 - params.max_liq_ratio):
                self.reserves_in = (prev_day.liq_usd * params.max_liq_ratio - prev_day.reserves * (1 - params.max_liq_ratio)) / params.max_liq_ratio
            else:
                self.reserves_in = 0
        
            #market dynamics
            self.net_flow = random.uniform(prev_day.liq_usd * prev_day.total_supply, prev_day.liq_usd * prev_day.total_demand)
            self.market_demand = params.demand_factor * short_sin(self.day, params.short_cycle) * long_sin(self.day, params.long_cycle, params.long_sin_offset)
            self.market_supply = params.supply_factor * short_cos(self.day, params.short_cycle) * long_cos(self.day, params.long_cycle, params.long_cos_offset)
            self.arb_factor = (prev_day.real_target / prev_day.price) ** ((self.day % params.short_cycle) / 5 - 1 )
            self.arb_demand = params.demand_factor * self.arb_factor
            self.arb_supply = params.supply_factor * (1 / (1 + self.arb_factor) - 1)

            if self.day >= params.short_cycle:
                unwind_date = self.day - (2 * self.day % params.short_cycle) - 1
                self.unwind_demand = (-1) * prev_arbs[unwind_date][1]
                self.unwind_supply = prev_arbs[unwind_date][0]
            else:
                self.unwind_demand = 0
                self.unwind_supply = 0
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)

            #liquidity
            if self.day % params.short_cycle == 0:
                self.liq_usd = (prev_day.real_target * prev_day.k)**(1/2) - self.reserves_in
                self.liq_ohm = self.k / self.liq_usd
                self.price = self.real_target
            else:
                self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_in
                self.liq_ohm = self.k / self.liq_usd
                self.price = self.liq_usd / self.liq_ohm

            #reserves
            self.reserves_out = self.liq_usd - self.net_flow - prev_day.liq_usd
            self.reserves = prev_day.reserves - self.reserves_out
            self.prev_reserves = prev_day.reserves


        self.treasury = self.liq_usd + self.reserves
        self.mcap = self.supply * self.price
        self.floating_supply = self.supply - self.liq_ohm
        self.floating_mcap = self.floating_supply * self.price
        self.liq_ratio = self.liq_usd / self.floating_mcap
        self.net_arb = self.arb_demand + self.arb_supply + self.unwind_demand + self.unwind_supply
        self.total_demand = self.market_demand + self.arb_demand + self.unwind_demand
        self.total_supply = self.market_supply + self.arb_demand + self.unwind_supply
        self.total_net = self.total_demand + self.total_supply


        #only to for reporting purposes (to check calculations)
        prev_lags['price'][1][self.day] = self.price
        prev_lags['target'][1][self.day] = self.real_target
        prev_lags['natural'][1][self.day] = self.natural_target


In [37]:
#traget price controller
def calc_price_target(short_cycle:int, prev_day:Day, prev_lags:Dict[int, Tuple[int, Dict[int, float]]]):
    if prev_day.day % short_cycle == 0:
        s = 0
        for key in prev_lags.keys():
          if key not in ('price', 'target', 'natural', 'avg'):
            days = prev_lags[key][1].keys()
            s += prev_lags[key][1][max(days)]
        avg_lag = s / (len(prev_lags.keys()) - 1)
        return (prev_day.natural_target + avg_lag) / 2
    else:
        return prev_day.real_target


def calc_natural_target(short_cycle:int, prev_day:Day):
    if prev_day.day % short_cycle == 0:
      if prev_day.day % (short_cycle * 4) == 0:
        return ((prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves) + prev_day.real_target) / 2
      else:
        return prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves
    return prev_day.natural_target


def calc_lag(day:int, params:ModelParams, prev_lags:Dict[int, Tuple[int, Dict[int, float]]], num_days:int=3):
    for key, values in prev_lags.items():
        if key not in ('price', 'target', 'natural', 'avg'):
          lag_days = values[0]
          if day > lag_days:
              if key == 'lag1':
                  if day > params.short_cycle:
                      s = prev_lags['price'][1][day-1]
                      for i in range(1, num_days):
                          s += prev_lags['price'][1][day - (i * lag_days)]
                      prev_lags[key][1][day] = s / num_days
                  else:
                    prev_lags[key][1][day] = values[1][day - 1]
              else:
                prev_lags[key][1][day] = prev_lags['lag1'][1][day - values[0]]
          else:
            prev_lags[key][1][day] = values[1][day - 1]


### __SET SCENARIO PARAMETERS__

In [38]:
#simulate scenario
params = ModelParams(horizon=720, max_liq_ratio=0.2, demand_factor=0.01, supply_factor=-0.01, short_cycle=90, long_cycle=1460, long_sin_offset=0.4375, long_cos_offset=0.875, initial_target=50)
arbs = {}
lags = {'price': (0, {1: 46}), 'target': (0, {1: 46}), 'natural': (0, {1: 46}), 'avg': (0, {1: 46}), 'lag1': (30, {1: params.initial_target}), 'lag2': (90, {1: params.initial_target}), 'lag3': (180, {1: params.initial_target}), 'lag4': (360, {1: params.initial_target})}

simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}

for i in range (2, params.horizon):
    simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])
    calc_lag(day=i, params=params, prev_lags=lags)

### __PLOT RESULTS__

In [39]:
#protocol variables
df = pd.DataFrame(columns = ['NetFlow', 'Price', 'Target', 'LiqUSD', 'LiqOHM', 'Reserves', 'ReservesIN', 'ReservesOUT', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio']) 
for day, data in simulation.items():
    df.loc[day] = [data.net_flow, data.price, data.real_target, data.liq_usd, data.liq_ohm, data.reserves, data.reserves_in, data.reserves_out, data.treasury, data.supply, data.mcap, data.floating_supply, data.floating_mcap, data.liq_ratio]


for col in df.columns:
    if col not in ('Price', 'Target'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        fig.show()

df[['Price','Target']].plot()


In [40]:
#market dynamics variables

market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb']) 
for day, data in simulation.items():
    market_df.loc[day] = [data.market_demand, data.market_supply, data.arb_factor, data.arb_demand, data.arb_supply, data.unwind_demand, data.unwind_supply, data.total_demand, data.total_supply, data.total_net, data.net_arb]

for col in market_df.columns:
    fig = market_df.plot(x=df.index, y=market_df[col])
    fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    for i in range (params.short_cycle, params.horizon, params.short_cycle):
        fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
    fig.show()

In [41]:
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 2*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset)]

df2.plot(y=df2.columns)

In [42]:
#variables related to price controller
price_target_df = pd.DataFrame(columns = ['Price', 'Target', 'Natural', 'Avg', 'Lag1', 'Lag2', 'Lag3', 'Lag4']) 
for key in lags.keys():
    for day, p in lags[key][1].items():
      price_target_df.at[day, key.capitalize()] = p

price_target_df.plot()